In [3]:
(defstruct person
  (birth-utime nil :type fixnum)
  (name nil :type string)
  (teknonym "" :type string)
  (sex nil :type (member #\m #\f))
  (children nil :type list))

PERSON

SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining COMMON-LISP-USER::COPY-PERSON in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining COMMON-LISP-USER::PERSON-P in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining (COMMON-LISP:SETF COMMON-LISP-USER::PERSON-BIRTH-UTIME) in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining COMMON-LISP-USER::PERSON-BIRTH-UTIME in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining (COMMON-LISP:SETF COMMON-LISP-USER::PERSON-NAME) in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining COMMON-LISP-USER::PERSON-NAME in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining (COMMON-LISP:SETF COMMON-LISP-USER::PERSON-TEKNONYM) in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining COMMON-LISP-USER::PERSON-TEKNONYM in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining (COMMON-LISP:SETF COMMON-LISP-USER::PERSON-SEX) in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining COMMON-LISP-USER::PERSON-SEX in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining (COMMON-LIS

In [44]:
(defun teknonym (s level name)
  (format nil "~v@{~A~:*~}~*~:[~;grand~]~:[father~;mother~] of ~A"
    (- level 2) "great-" (> level 1) (char= s #\f) name))

TEKNONYM

SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining COMMON-LISP-USER::TEKNONYM in DEFUN


In [45]:
(teknonym #\f 4 "Anna")

"great-great-grandmother of Anna"

In [39]:
(defun teknonymize (tree)
  (declare (type person tree))
  (labels
    ((descend (n)
      (declare (type person tree))
      (if (person-children n)
        (multiple-value-bind (level d) (descend (first (person-children n)))
          (dolist (c (cdr (person-children n)))
            (multiple-value-bind (clevel cd) (descend c)
              (when (or (< level clevel)
                        (and (= level clevel)
                             (< (person-birth-utime cd) (person-birth-utime d))))
                  (setf level clevel d cd))))
          (setf (person-teknonym n) (teknonym (person-sex n) level (person-name d)))
          (values (1+ level) d))
        (values 1 n))))
    (descend tree)
    (values)))

TEKNONYMIZE

SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining COMMON-LISP-USER::TEKNONYMIZE in DEFUN


In [40]:
(defparameter *t*
  (make-person :birth-utime 1000 :name "Abdul" :sex #\m
    :children
    (list (make-person :birth-utime 800 :name "Sarah" :sex #\f))))

*T*

In [43]:
*t*

#S(PERSON
   :BIRTH-UTIME 1000
   :NAME "Abdul"
   :TEKNONYM "mother of Sarah"
   :SEX #\m
   :CHILDREN (#S(PERSON
                 :BIRTH-UTIME 800
                 :NAME "Sarah"
                 :TEKNONYM ""
                 :SEX #\f
                 :CHILDREN NIL)))

In [42]:
(teknonymize *t*)